In [1]:
# news to vector
# ptt content to vector
# cal cosine similarity
# keep update top 10 similary
# write similary news url: update db table

In [2]:
import pymysql
from datetime import datetime, timedelta
import pandas as pd
import warnings
warnings.filterwarnings('ignore') 
import jieba
import json
from tqdm.notebook import tqdm
from os import listdir

In [3]:
db_settings = {
        "host": "localhost",
        "port": 3306,
        "user": "root",
        "password": "ai52004800",
        "db": "exp",
        "charset": "utf8"
        }


db_timeformat = "%Y-%m-%d %H:%M:%S"

conn = pymysql.connect(**db_settings)

In [4]:
time_exp = datetime.strptime("2021-07-06 12:00:00", "%Y-%m-%d %H:%M:%S")
time_start = time_exp - timedelta(hours=1)
time_end = time_exp + timedelta(hours=1)

In [5]:
sql = "SELECT * FROM `ptt_content` WHERE `tp` BETWEEN '%s' AND '%s'" %(time_start,time_end)
sql

"SELECT * FROM `ptt_content` WHERE `tp` BETWEEN '2021-07-06 11:00:00' AND '2021-07-06 13:00:00'"

In [6]:
# 讀ptt 資料
with conn.cursor() as cursor:
    sql = "SELECT * FROM `ptt_content` WHERE `tp` BETWEEN '%s' AND '%s' AND push_count>100" %(time_start, time_end)
    cursor.execute(sql)
    r = cursor.fetchall()
    ptt_content_df = pd.DataFrame(list(r),
                              columns=['url', 'title', 'content', 'tp', 'author', 'board', 'ip', 'country', 'push_count'])

In [7]:
len(ptt_content_df)

31

In [8]:
from gensim.models.keyedvectors import KeyedVectors

model_path = "w2v.model.bin"
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
# How to use bin(model)?

In [9]:
for f in listdir("./predefined/dictionary/"):
    jieba.load_userdict("./predefined/dictionary/"+f)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\AICENT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.795 seconds.
Prefix dict has been built successfully.


In [10]:
stop_wd = set()
with open("./predefined/stopwords_all.json", 'r', encoding='utf8') as f:
    for w in json.load(f):
        stop_wd.add(w)
        
with open("./predefined/stopwords_news.json", 'r', encoding='utf8') as f:
    for w in json.load(f):
        stop_wd.add(w)

In [11]:
import numpy as np

class DocSim:
    #https://github.com/v1shwa/document-similarity
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.strip().replace("\n","")
        words = jieba.cut(doc)
        words = [word for word in words if word != ' ' and  word not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0, top=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

#         if isinstance(target_docs, str):
#             target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        
        for doc in target_docs:
            target_vec = self.vectorize(doc[1])
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "url": doc[0], "title": doc[2],"keywords": doc[3]})
#                 results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)
        if top>0:
            return results[:top]

        return results

In [12]:
ds = DocSim(w2v_model)

In [16]:
%%time
from pprint import pprint
for index in range(len(ptt_content_df)):
    if index>5 and index<15:
        source_doc = ptt_content_df.content[index]
        src_time = ptt_content_df.tp[index]

        time_start = src_time - timedelta(hours=12)
        time_end = src_time + timedelta(hours=12)

        with conn.cursor() as cursor:
            sql = "SELECT * FROM `news` WHERE `published_time` BETWEEN '%s' AND '%s'" %(time_start, time_end)
            cursor.execute(sql)
            r = cursor.fetchall()
            target_docs = []
            for row in r:
                target = [row[0], row[2], row[1], row[5]]
                target_docs.append(target)

        # This will return target docs with similarity score
        sim_scores = ds.calculate_similarity(source_doc, target_docs, top=5)
        print("========================================")
        print("時間區間內所有新聞: ", len(target_docs))
        print("source_doc url :",ptt_content_df.url[index])
        print("\n=============前五篇相關新聞================\n")
        pprint(sim_scores)

時間區間內所有新聞:  2322
source_doc url : M.1625542259.A.0F3

=============前五篇相關新聞================

[{'keywords': '郵局,公務員,PTT,薪水',
  'score': 0.8854719,
  'title': '郵局加薪讓年薪多8萬 但網友談工作情況「超血汗的」',
  'url': 'https://udn.com/news/story/7269/5582524'},
 {'keywords': '紓困貸款,紓困10萬,航海王,航運股,韭菜',
  'score': 0.88333535,
  'title': '10萬紓困貸款到手！夢想閉眼「歐印」航運股當航海王網怒罵賭博當韭菜',
  'url': 'https://tw.appledaily.com/property/20210706/GWEEWBTPU5FPXKEXHKZ3CP6YQI/'},
 {'keywords': 'K律師論點,炒房,存房,潔哥',
  'score': 0.8585314,
  'title': 'K律師論點｜潔哥挨批炒房無殼蝸牛真正對手是這一族',
  'url': 'https://tw.appledaily.com/local/20210706/VZQ5EKHRLJB75IVMPUNRNUXJMM/'},
 {'keywords': '網友,月薪,薪水,年終,獎金',
  'score': 0.8547201,
  'title': '月薪上看60K卻為這一點想轉行 網嗆：被虐狂嗎？',
  'url': 'https://www.chinatimes.com/realtimenews/20210706002764-260402'},
 {'keywords': '潔哥,李秉潔,預售屋,酸民,換房',
  'score': 0.8533366,
  'title': '2房換3房再換大4房！潔哥被質疑賣鮑她親曝錢怎麼來的',
  'url': 'https://tw.appledaily.com/life/20210706/53WASATGEVACHENXBEINR7DM2Y/'}]
時間區間內所有新聞:  2320
source_doc url : M.1625542643